In [2]:
import itertools
import warnings

import pandas as pd
from tqdm.auto import tqdm
from utils import (
    ExperimentSchema,
    OptimizationSchema,
    run_experiment,
)

In [3]:
# load ARD data
df_s1 = pd.read_parquet("./data/s1_ard.pq")
df_s2 = pd.read_parquet("./data/s2_ard.pq")
df_mw = pd.read_parquet("./data/s2_mowing.pq")

In [10]:
exp_settings = {
    "s1_var": ["VV", "VH", "RVI"],
    "normalize": [True, False],
    "group_size": [1, 2, 4],
    "kernel_name": ["ICM_Mat32", "ICM_Bias+Mat32", "LCM_Bias+Mat32"],
}


def product_dict(settings):
    keys = settings.keys()
    vals = settings.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))


base_config = dict(
    n_groups=2,
    opt_config=OptimizationSchema(num_restarts=1, num_processes=1, optimizer="org-bfgs"),
    save_to="./results/",
)

warnings.filterwarnings("ignore")
configs = [ExperimentSchema(**base_config, **settings) for settings in list(product_dict(exp_settings))]
for config in tqdm(configs):
    _ = run_experiment(df_s1, df_s2, config)

  0%|          | 0/54 [00:00<?, ?it/s]

In [4]:
# run_experiment(df_s1, df_s2, ExperimentSchema(**{**config.dict(), "group_size": 1, "n_groups": 1}))